In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import plotly.io as pio

In [2]:
df=pd.read_excel("../n_obs/SIO_TEMP_20230501.xls",skiprows=47, header=None) #change excel filename
df = df.rename(columns={0: 'year',
                        1: 'month',
                        2: 'day',
                        5: 'sst'})
df = df[(df['year'] > 1916) & (df['year'] < 2023)] #exclude partial years 1916 & 2023
df['date'] = pd.to_datetime(df[['year','month','day']])
df

,year,month,day,3,4,sst,6,7,8,date
114,1917,1,1,NaN,NaN,NaN,0,NaN,NaN,1917-01-01
115,1917,1,2,NaN,NaN,11.5,0,NaN,NaN,1917-01-02
116,1917,1,3,NaN,NaN,11.9,0,NaN,NaN,1917-01-03
117,1917,1,4,NaN,NaN,11.5,0,NaN,NaN,1917-01-04
118,1917,1,5,NaN,NaN,11.7,0,NaN,NaN,1917-01-05
...,...,...,...,...,...,...,...,...,...,...
38825,2022,12,27,1440.0,0.0,14.7,0,14.6,0.0,2022-12-27
38826,2022,12,28,1320.0,0.0,15.1,0,14.9,0.0,2022-12-28
38827,2022,12,29,1430.0,0.0,14.9,0,14.9,0.0,2022-12-29
38828,2022,12,30,1719.0,0.0,15.3,0,15.1,0.0,2022-12-30


In [3]:
daily_climatology = df.groupby(['month','day']).agg(['min','mean','max','count'])
all_hi = daily_climatology[('sst','max')]
all_lo = daily_climatology[('sst','min')]
intraannual_cycle = daily_climatology[('sst','mean')]

def set_yearly_data(year): #TODO fix leap day offset issue
    global year_dates, n_years, year_sst, daily_hi_days, daily_hi_ssts, daily_lo_days, daily_lo_ssts
    year_dates = [datetime(year,date[0],date[1]) for date in df[df.year == year].groupby(['month','day']).first().index.values]
    n_years = str(daily_climatology[('sst','count')].max())
    year_sst = df[df.year == year].sst
    daily_hi_days = []
    daily_hi_ssts = []
    daily_lo_days = []
    daily_lo_ssts = []
    for i, sst in enumerate(year_sst):
        if sst >= all_hi.iloc[i]: 
            daily_hi_days.append(year_dates[i])
            daily_hi_ssts.append(sst)
        if sst >= all_hi.iloc[i]: 
            daily_lo_days.append(year_dates[i])
            daily_lo_ssts.append(sst)

In [4]:
year = 2015
set_yearly_data(2015)

In [5]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(
    x=year_dates, 
    y=all_hi, 
    mode='lines',
    line=dict(
        width=1,
        color='lightgray'),
    name='Daily High SST (from ' + n_years + ' years)'),
    secondary_y=False)
fig.add_trace(go.Scatter(
    x=year_dates, 
    y=all_lo, 
    mode='lines', 
    line=dict(
        width=1,
        color='lightgray'),
    name='Daily Low SST (from ' + n_years + ' years)', #TODO make custom legend entry for "All Observed Temp" shading
    fill='tonexty'),
    secondary_y=False)

fig.add_trace(go.Scatter( #plot the selected year's data
    x=year_dates,
    y=year_sst,
    name='SST from '+str(year),
    mode='lines',
    line=dict(
        width=2,
        color='#404040'),
    showlegend=True),
    secondary_y=False)

fig.add_trace(go.Scatter( #plot the intra-annual cycle
    x=year_dates,
    y=intraannual_cycle,
    name='Intra-anual Cycle (from ' + n_years + ' years)',
    mode='lines',
    line=dict(
        width=1,
        color='grey',
        dash='dash'),
    showlegend=True),
    secondary_y=False)

fig.add_trace(go.Scatter( #plot the daily los
    x=daily_lo_days,
    y=daily_lo_ssts,
    name='Day Record Low (' + str(year) + ')',
    mode='markers',
    marker=dict(
        size=5,
        color='blue'),
    showlegend=True),
    secondary_y=False)

fig.add_trace(go.Scatter( #plot the daily los
    x=daily_hi_days,
    y=daily_hi_ssts,
    name='Day Record High (' + str(year) + ')',
    mode='markers',
    marker=dict(
        size=5,
        color='red'),
    showlegend=True),
    secondary_y=False)

#TODO remove year from x axis, make it so you can see every month
fig.update_yaxes(dict(title='Sea Surface Temperature (°C)', range=[all_lo.min() - 0.5, all_hi.max() + 0.5], showgrid=False, side='left'), secondary_y=False)
fig.update_layout(title_text='How does '+str(year)+' SST at SIO compare to others?',
    plot_bgcolor='white',
    hovermode='x unified') #TODO make box outline red or blue if its a daily record
pio.write_html(fig,'fig/'+'interactive_bottomplot_SIO.html') #TODO fix aspect ratio

In [6]:
fig